In [34]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.python import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import SimpleRNN , Input,Flatten, LSTM
from tensorflow.keras import backend as K
from keras import initializers
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

In [35]:
opened = []

csv_path = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/CSV/'
masterlist = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/ALL_STATIONS_ALL_MONTHS_PRECIP.csv'

# Load the master list of stations
total_df = pd.read_csv(masterlist)
print(total_df.head())

   Unnamed: 0     Date       Q       SWE   SWE_scaled  GRDC_No  SHP      Area  \
0           0  2018-08  24.526  2.087791  1809042.021  1159100    1  866486.0   
1           1  2018-09  31.372  1.835435  1590378.590  1159100    1  866486.0   
2           2  2018-10  19.572  1.976332  1712463.856  1159100    1  866486.0   
3           3  2018-11   7.349  1.633273  1415208.035  1159100    1  866486.0   
4           4  2018-12  13.824  1.782850  1544814.625  1159100    1  866486.0   

   Latitude  Avg Slope  Max Slope    Aridity    Precip             P  
0 -28.75799   0.745665  22.007082  907.27856  3.059264  2.298018e+05  
1 -28.75799   0.745665  22.007082  907.27856  2.724767  2.989010e+05  
2 -28.75799   0.745665  22.007082  907.27856  8.292236  8.098043e+05  
3 -28.75799   0.745665  22.007082  907.27856  3.910201  1.301366e+06  
4 -28.75799   0.745665  22.007082  907.27856  9.718854  1.729622e+06  


In [36]:
landcover = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/CSV/ALL_STATIONS_LANDCOVER.csv'
landcover_df = pd.read_csv(landcover)

# Ensure number and GRDC_No are of the same type
total_df['GRDC_No'] = total_df['GRDC_No'].astype(int)
landcover_df['number'] = landcover_df['number'].astype(int)

# Initialize empty lists for new columns
new_columns = {col: [] for col in landcover_df.columns if col != 'number'}

# Loop through total_df and match with landcover_df
for grdc_no in total_df['GRDC_No']:
    match = landcover_df[landcover_df['number'] == grdc_no]
    
    if not match.empty:
        for col in new_columns:
            new_columns[col].append(match[col].values[0])  # Append matched value
    else:
        for col in new_columns:
            new_columns[col].append(None)  # Append NaN if no match found

# Convert lists to a DataFrame and concatenate with total_df
new_data = pd.DataFrame(new_columns)
total_df = pd.concat([total_df, new_data], axis=1)

print(total_df.head())

   Unnamed: 0     Date       Q       SWE   SWE_scaled  GRDC_No  SHP      Area  \
0           0  2018-08  24.526  2.087791  1809042.021  1159100    1  866486.0   
1           1  2018-09  31.372  1.835435  1590378.590  1159100    1  866486.0   
2           2  2018-10  19.572  1.976332  1712463.856  1159100    1  866486.0   
3           3  2018-11   7.349  1.633273  1415208.035  1159100    1  866486.0   
4           4  2018-12  13.824  1.782850  1544814.625  1159100    1  866486.0   

   Latitude  Avg Slope  ...        11        12        13        14        15  \
0 -28.75799   0.745665  ...  0.725055  0.678026  0.634266  0.625967  0.632506   
1 -28.75799   0.745665  ...  0.725055  0.678026  0.634266  0.625967  0.632506   
2 -28.75799   0.745665  ...  0.725055  0.678026  0.634266  0.625967  0.632506   
3 -28.75799   0.745665  ...  0.725055  0.678026  0.634266  0.625967  0.632506   
4 -28.75799   0.745665  ...  0.725055  0.678026  0.634266  0.625967  0.632506   

         16       17      

In [37]:
soiltype = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/CSV/ALL_STATIONS_SOILTYPE.csv'
soiltype_df = pd.read_csv(soiltype)
# print(soiltype_df.head())
# Ensure number and GRDC_No are of the same type
total_df['GRDC_No'] = total_df['GRDC_No'].astype(int)
soiltype_df['number'] = soiltype_df['number'].astype(int)

# Initialize empty lists for new columns
new_columns = {col: [] for col in soiltype_df.columns if col != 'number'}

# Loop through total_df and match with landcover_df
for grdc_no in total_df['GRDC_No']:
    match = soiltype_df[soiltype_df['number'] == grdc_no]
    
    if not match.empty:
        for col in new_columns:
            new_columns[col].append(match[col].values[0])  # Append matched value
    else:
        for col in new_columns:
            new_columns[col].append(None)  # Append NaN if no match found

# Convert lists to a DataFrame and concatenate with total_df
new_data = pd.DataFrame(new_columns)
total_df = pd.concat([total_df, new_data], axis=1)

print(total_df.head())

   Unnamed: 0     Date       Q       SWE   SWE_scaled  GRDC_No  SHP      Area  \
0           0  2018-08  24.526  2.087791  1809042.021  1159100    1  866486.0   
1           1  2018-09  31.372  1.835435  1590378.590  1159100    1  866486.0   
2           2  2018-10  19.572  1.976332  1712463.856  1159100    1  866486.0   
3           3  2018-11   7.349  1.633273  1415208.035  1159100    1  866486.0   
4           4  2018-12  13.824  1.782850  1544814.625  1159100    1  866486.0   

   Latitude  Avg Slope  ...         7         8         9        10       11  \
0 -28.75799   0.745665  ...  1.132788  1.497797  5.198238  0.453115  0.98175   
1 -28.75799   0.745665  ...  1.132788  1.497797  5.198238  0.453115  0.98175   
2 -28.75799   0.745665  ...  1.132788  1.497797  5.198238  0.453115  0.98175   
3 -28.75799   0.745665  ...  1.132788  1.497797  5.198238  0.453115  0.98175   
4 -28.75799   0.745665  ...  1.132788  1.497797  5.198238  0.453115  0.98175   

        12   13   14   15   16  

In [38]:
q = total_df['Q']
swe = total_df['SWE']
scaled = total_df['SWE_scaled']
area = total_df['Area']
station = total_df['GRDC_No']
lat = total_df['Latitude']
avg_slope = total_df['Avg Slope']
max_slope = total_df['Max Slope']
precip = total_df['P']

In [39]:
lc_types = total_df.iloc[:, 14:-16]
soil_types = total_df.iloc[:, -16:]

# lc_types = lc_types/100
# soil_types = soil_types/100

print(lc_types)
print(soil_types)

              1          2          3         4         5         6         7  \
0      6.136561  60.524866  17.803458  3.554480  1.860924  1.402201  1.166174   
1      6.136561  60.524866  17.803458  3.554480  1.860924  1.402201  1.166174   
2      6.136561  60.524866  17.803458  3.554480  1.860924  1.402201  1.166174   
3      6.136561  60.524866  17.803458  3.554480  1.860924  1.402201  1.166174   
4      6.136561  60.524866  17.803458  3.554480  1.860924  1.402201  1.166174   
...         ...        ...        ...       ...       ...       ...       ...   
23053  5.348157  14.675249  16.582797  9.654769  5.874781  4.628438  4.253950   
23054  5.348157  14.675249  16.582797  9.654769  5.874781  4.628438  4.253950   
23055  5.348157  14.675249  16.582797  9.654769  5.874781  4.628438  4.253950   
23056  5.348157  14.675249  16.582797  9.654769  5.874781  4.628438  4.253950   
23057  5.348157  14.675249  16.582797  9.654769  5.874781  4.628438  4.253950   

              8         9  

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU

lc_in = lc_types.idxmax(axis=1)
soil_in = soil_types.idxmax(axis=1)
# print(lc_class)

# lat_in = (lat*np.pi)/180
# Convert q and swe to NumPy arrays
scaled_in = scaled.to_numpy()  # Assume scaled is a pandas Series
area_in = area.to_numpy()      # Assume area is a pandas Series
lat_in = lat.to_numpy()
avg_slope_in = avg_slope.to_numpy()
# avg_slope_in = (avg_slope_in*np.pi)/180
max_slope_in = max_slope.to_numpy()
# max_slope_in = (max_slope_in*np.pi)/180
# avg_p_in = precip.to_numpy()
y = q.to_numpy()            # Target variable

lc_in = [int(item) for item in lc_in]
soil_in = [int(item) for item in soil_in]

# Convert q and swe to NumPy arrays
# scaled_in = scaled_in/max(scaled_in)  # Assume scaled is a pandas Series
# area_in = area_in/max(area_in)      # Assume area is a pandas Series
# lat_in = lat_in/180
# avg_slope_in = avg_slope_in/180
# max_slope_in = max_slope_in/180
# avg_aridity_in = avg_aridity_in/max(avg_aridity_in)
# y = y/max(y)            # Target variable
# lc_in = lc_types/max(lc_types)
# soil_in = soil_types/max(soil_types)

# Step 1: Replace zeros with a small value
scaled_in[scaled_in == 0] = 1e-9
area_in[area_in == 0] = 1e-9
lat_in[lat_in == 0] = 1e-9
avg_slope_in[avg_slope_in == 0] = 1e-9
max_slope_in[max_slope_in == 0] = 1e-9
avg_p_in[avg_p_in == 0] = 1e-9
y[y == 0] = 1e-9

# Step 2: Combine features into a 2D array
X = np.column_stack((scaled_in, area_in, lat_in, avg_slope_in, max_slope_in, avg_p_in, lc_in, soil_in))  # Shape will be (num_samples, 2)
print(X[0])

[ 1.80904202e+06  8.66486000e+05 -2.87579900e+01  7.45665431e-01
  2.20070820e+01  2.29801800e+05  2.00000000e+00  3.00000000e+00]


In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 1. Standardize the data (each column)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Apply PCA
pca = PCA(n_components=2)  # for example, reduce to 2 components
X_pca = pca.fit_transform(X_scaled)


In [49]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(X_scaled)
pca_X = pca.transform(X_scaled)

# Create a new DataFrame with the principal components
pca_df = pd.DataFrame(data=pca_X, columns=['pc1','pc2','pc3'])

# Display the explained variance ratio
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")

# To see the importance of each feature for the principal components
# The components_ attribute holds the loadings (how much each feature contributes to each principal component)
loadings = pd.DataFrame(pca.components_, columns=['SWE','Area','Latitude','Avg Slope','Max Slope','Precip','Land Cover','Soil Texture'], index=[f'PC{i+1}' for i in range(pca.n_components_)])
# print(loadings)

# Optionally, you can sort the loadings to identify the most important features
# Example: Sort by PC1 loadings
print("Top features for PC1:")
print(loadings.loc['PC3'].abs().sort_values(ascending=False))

Explained variance ratio: [0.28460557 0.22839447 0.15484371]
Top features for PC1:
Precip          0.625889
Avg Slope       0.497800
Land Cover      0.419586
Max Slope       0.287268
Soil Texture    0.211896
Latitude        0.165124
Area            0.160052
SWE             0.064032
Name: PC3, dtype: float64


In [43]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assume X is your original DataFrame
feature_names = ['SWE','Area','Latitude','Avg Slope','Max Slope','Precip','Land Cover','Soil Texture']

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA
pca = PCA(n_components=3)
pca_X = pca.fit_transform(X_scaled)

# Loadings matrix (PCs × features)
loadings = pd.DataFrame(
    pca.components_, 
    columns=feature_names, 
    index=[f'PC{i+1}' for i in range(pca.n_components_)]
)

# Get top feature for each PC
top_features = loadings.abs().idxmax(axis=1)

# Combine with explained variance ratio
explained_variance_summary = pd.DataFrame({
    'Top Feature': top_features,
    'Explained Variance Ratio': pca.explained_variance_ratio_
})

print(explained_variance_summary)


    Top Feature  Explained Variance Ratio
PC1   Max Slope                  0.284606
PC2         SWE                  0.228394
PC3      Precip                  0.154844


In [44]:
U, D, VT = np.linalg.svd(X_scaled, full_matrices=False)
print(U)

[[-0.01020457  0.00473088  0.01088511 ... -0.00106247  0.00578572
   0.03134196]
 [-0.00999381  0.00448557  0.01073718 ... -0.00127046  0.00610359
   0.03319683]
 [-0.00997852  0.00494281  0.00996465 ... -0.00241469  0.00592923
   0.03234889]
 ...
 [-0.00356639 -0.0050305  -0.0026025  ...  0.0066166   0.0074925
  -0.00076681]
 [-0.00356676 -0.00503583 -0.00259299 ...  0.00663068  0.00749434
  -0.00075815]
 [-0.00356429 -0.00502693 -0.00261361 ...  0.00660025  0.00749316
  -0.00076079]]


In [45]:
import matplotlib.pyplot as plt

residuals = y_test - model.predict(X_test)
plt.scatter(y_test, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Actual Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()


NameError: name 'y_test' is not defined

In [ ]:
# Plot y_test and y_pred as a time series
plt.figure(figsize=(12, 6))
plt.plot(np.arange(len(y_test)), y_test*max(y), label="Actual Values (y_test)", color="blue", alpha=0.7)
plt.plot(np.arange(len(y_pred)), y_pred*max(y), label="Predicted Values (y_pred)", color="red", alpha=0.7)
plt.title("Actual vs Predicted Values of Test Dataset")
plt.xlabel("Test Dataset")
plt.ylabel("Streamflow [m${^3}$/s]")
plt.xlim(0,len(y_test))
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot y_test and y_pred as a time series
plt.figure(figsize=(12, 6))
plt.plot((np.arange(len(y_test)))[0:500], (y_test*max(y))[0:500], label="Actual Values (y_test)", color="blue", alpha=0.7)
plt.plot((np.arange(len(y_pred)))[0:500], (y_pred*max(y))[0:500], label="Predicted Values (y_pred)", color="red", alpha=0.7)

# Add titles and labels
plt.title("Actual vs Predicted Values of Test Dataset (element 0 to 500)")
plt.xlabel("Test Dataset")
plt.ylabel("Streamflow [m${^3}$/s]")
# plt.xlim(0,len(y_test))
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(scaled,q)
plt.title('Raw Data - All Basins')
plt.xlabel('Surface Water Extent Area [km${^2}$]')
plt.ylabel('Streamflow [m${^3}$/s]')
# plt.savefig('global/scratch/users/arvalcarcel/CSMUB/RESULTS/CNN/rawdata.png')
plt.show()

In [ ]:
plt.scatter(area,q,color='g')
plt.title('Raw Data - All Basins')
plt.xlabel('Total Basin Area [km${^2}$]')
plt.ylabel('Streamflow [m${^3}$/s]')
# plt.savefig('global/scratch/users/arvalcarcel/CSMUB/RESULTS/CNN/rawdata.png')
plt.show()

In [ ]:
plt.scatter(lat,q,color='r')
plt.title('Raw Data - All Basins')
plt.xlabel('Latitude [deg]')
plt.ylabel('Streamflow [m${^3}$/s]')
# plt.savefig('global/scratch/users/arvalcarcel/CSMUB/RESULTS/CNN/rawdata.png')
plt.show()

In [ ]:
plt.scatter(max_slope,q,color='purple')
plt.title('Raw Data - All Basins')
plt.xlabel('Max Slope [deg]')
plt.ylabel('Streamflow [m${^3}$/s]')
# plt.savefig('global/scratch/users/arvalcarcel/CSMUB/RESULTS/CNN/rawdata.png')
plt.show()

In [ ]:
# lc_class = lc_types.idxmax(axis=1)
plt.scatter(lc_class,q,color='orange')
plt.title('Raw Data - All Basins')
plt.xlabel('Land Cover Type')
plt.ylabel('Streamflow [m${^3}$/s]')
# plt.savefig('global/scratch/users/arvalcarcel/CSMUB/RESULTS/CNN/rawdata.png')
plt.show()

In [ ]:
# soil_class = soil_types.idxmax(axis=1)

plt.scatter(soil_class,q,color='darkorange')
plt.title('Raw Data - All Basins')
plt.xlabel('Soil Texture')
plt.ylabel('Streamflow [m${^3}$/s]')
# plt.savefig('global/scratch/users/arvalcarcel/CSMUB/RESULTS/CNN/rawdata.png')
plt.show()

In [ ]:
grouped = total_df.groupby('Area')['Q'].mean()

# Extract unique areas and their corresponding average Q values
area_unique = grouped.index
q_avg = grouped.values

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(area_unique, q_avg, color='g')
plt.xlabel('Total Basin Area')
plt.ylabel('Average Streamflow')
plt.title('Average Streamflow vs Total Basin Area')
plt.grid(True)
plt.show()

In [ ]:
grouped = total_df.groupby('Latitude')['Q'].mean()

# Extract unique areas and their corresponding average Q values
lat_unique = grouped.index
q_avg = grouped.values

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(lat_unique, q_avg,color='red')
plt.xlabel('Latitude')
plt.ylabel('Average Streamflow')
plt.title('Average Streamflow vs Basin Latitude')
plt.grid(True)
plt.show()

In [ ]:
filename = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/CNN/rnn_plot.png'
# Ensure the directory exists
os.makedirs(os.path.dirname(filename), exist_ok=True)
from tensorflow.keras.utils import plot_model

# Plot and save the model
plot_model(model, to_file=filename, show_shapes=True, show_layer_names=True)

from IPython.display import Image
Image(filename)